In [1]:
!pip install qdrant_client

  Using cached qdrant_client-1.3.1-py3-none-any.whl (132 kB)
  Using cached grpcio_tools-1.56.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.6 MB)
  Using cached httpx-0.24.1-py3-none-any.whl (75 kB)
  Using cached portalocker-2.7.0-py2.py3-none-any.whl (15 kB)
  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Using cached urllib3-1.26.16-py2.py3-none-any.whl (143 kB)
  Using cached httpcore-0.17.3-py3-none-any.whl (74 kB)
  Using cached h2-4.1.0-py3-none-any.whl (57 kB)
  Using cached hyperframe-6.0.1-py3-none-any.whl (12 kB)
  Using cached hpack-4.0.0-py3-none-any.whl (32 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.3
    Uninstalling urllib3-2.0.3:
      Successfully uninstalled urllib3-2.0.3
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typ

In [2]:
from qdrant_client import QdrantClient

In [3]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
import os


qdrant_host = os.getenv('QDRANT_HOST') or '127.0.0.1'
qdrant_port = os.getenv('QDRANT_PORT') or "6333"

client = QdrantClient(qdrant_host, port=qdrant_port)
client.recreate_collection(
    collection_name="test_collection",
    vectors_config=VectorParams(size=4, distance=Distance.DOT),
)

True

In [5]:
collection_info = client.get_collection(collection_name="test_collection")
collection_info

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=0, indexed_vectors_count=0, points_count=0, segments_count=2, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=4, distance=<Distance.DOT: 'Dot'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=1, replication_factor=1, write_consistency_factor=1, on_disk_payload=True), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None), payload_schema={})

In [6]:
from qdrant_client.http.models import PointStruct

operation_info = client.upsert(
    collection_name="test_collection",
    wait=True,
    points=[
        PointStruct(id=1, vector=[0.05, 0.61, 0.76, 0.74], payload={"city": "Berlin"}),
        PointStruct(id=2, vector=[0.19, 0.81, 0.75, 0.11], payload={"city": ["Berlin", "London"]}),
        PointStruct(id=3, vector=[0.36, 0.55, 0.47, 0.94], payload={"city": ["Berlin", "Moscow"]}),
        PointStruct(id=4, vector=[0.18, 0.01, 0.85, 0.80], payload={"city": ["London", "Moscow"]}),
        PointStruct(id=5, vector=[0.24, 0.18, 0.22, 0.44], payload={"count": [0]}),
        PointStruct(id=6, vector=[0.35, 0.08, 0.11, 0.44]),
    ]
)

In [7]:
search_result = client.search(
    collection_name="test_collection",
    query_vector=[0.2, 0.1, 0.9, 0.7], 
    limit=3
)

In [8]:
search_result

[ScoredPoint(id=4, version=0, score=1.362, payload={'city': ['London', 'Moscow']}, vector=None),
 ScoredPoint(id=1, version=0, score=1.273, payload={'city': 'Berlin'}, vector=None),
 ScoredPoint(id=3, version=0, score=1.208, payload={'city': ['Berlin', 'Moscow']}, vector=None)]